In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())  


True

In [ ]:
def get_chat_completion(prompt, model) ->str: 
    response = llm.invoke(prompt) 
    return response.content 

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile",
    temperature=0
)
query=""

In [ ]:
FirstPrompt = f"""

"""


In [ ]:
SecondPrompt = f"""      
You are an expert SQLite developer and data analyst specializing in querying complex, event-driven systems. Your task is to convert a natural language data request into a single, syntactically correct SQLite query based on a provided database schema and detailed event definitions.

**1. Database Schema**

You MUST adhere strictly to the following SQLite schema. Do not invent table or column names.

```sql
{schema}

    

2. Event Logic Context

This context explains the meaning behind the tables and the business logic of the events. Use this to understand the relationships between tables and the purpose of the data. For example, to find information about "unsafe velocity," you must use the PLUNGER_UNSAFE_VELOCITY_EVENTS table and join it with related tables.
code Code
IGNORE_WHEN_COPYING_START
IGNORE_WHEN_COPYING_END

      
{event_details}

    

3. User's Data Request

The user needs data for the following scenario. Your query must retrieve exactly what is being asked.

    Description: "{description}"

4. Your Task & Constraints

    Analyze the User's Data Request.

    Use the Database Schema and Event Logic Context to identify all the necessary tables and columns.

    Construct the correct JOIN clauses to link the tables based on their foreign key relationships.

    Translate any time-based constraints (e.g., "last 7 days," "yesterday") into appropriate SQLite WHERE clauses. Use standard SQLite date functions like datetime('now', '-7 days').

    You must output ONLY the raw SQLite query.

    Do not include any explanations, comments, or markdown formatting like sql ... . Your entire output should be the executable query itself."""

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(FirstPrompt) 

In [5]:
def extract_code_from_markdown(llm_output: str) -> str:
    if llm_output.startswith("```"):
        return "\n".join(
            line for line in llm_output.splitlines()
            if not line.strip().startswith("```")
        )
    return llm_output

user_query = "give me the latest 5 reading of arival speed"
message = prompt_template.format_messages(query=user_query)
resp = get_chat_completion(message, llm)
extracted_code = extract_code_from_markdown(resp)
extracted_code

"import pandas as pd\nimport glob\n\n# Load Arrival Speed data\narrival_speed_file = glob.glob('./La Vista 1H/Arrival Speed*.csv')[0]\narrival_speed_df = pd.read_csv(arrival_speed_file)\n\n# Ensure timestamp is in datetime format\narrival_speed_df['timestamp'] = pd.to_datetime(arrival_speed_df['timestamp'])\n\n# Sort by timestamp in descending order and get the latest 5 readings\nlatest_5_readings = arrival_speed_df.sort_values(by='timestamp', ascending=False).head(5)\n\nprint(latest_5_readings)"